# Cognitive Distance Methods

This notebook will accompany the paper 'Towards a Geospatial Model of Cognitive Distance in Urban Spaces: An Objective Measure of Navigibility' by Ed Manley, Gabriele Filomena, and Panos Mavros.

The notebook contains the methods used to generate estimates of cognitive distance from conventional GIS data. It also demonstrates the implementation of these methods in estimating cognitive distance in 16 cities.

## Libraries

In [64]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
import math
import requests
import numpy as np
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import shape
import matplotlib.pyplot as plt 


%matplotlib inline
plt.style.use('ggplot')
ox.config(log_file=True, log_console=True, use_cache=True)

In [62]:
import urbanFormPy as up

## Methods

Each method below simulates the effect of one or more types of cognitive bias in distance perception, as highlighted in the paper. The methods are implemented later in generating estimates for case study cities.

### Landmarks

In [ ]:
def detect_landmark(city, road_network, distance_from_center = 1000):
    
    # return GeoDataFrame with buildings and landmark scores
    
    buildings_gdf = up.get_buildings_from_OSM([city], distance_from_center)[0] #works with list of cities
    
    ox.projection.project_gdf(road_network)
    # structural score
    buildings_gdf = up.structural_score(buildings_gdf, road_network, max_expansion_distance = 300, 
                                            distance_along = 50, radius = 150)

    # visual score: not really computed at the moment as in most of the cases height information is not provided in OSM
    buildings_gdf = up.visual_score(buildings_gdf)
    
    # culturalscore:
    buildings_gdf = up.cultural_score_from_OSM(buildings_gdfs)
    
    # pragmatic score
    """
    The "radius" parameter indicates the extension of the area that is used to compute the score of a building, on the basis of
    its land-use's frequency, in an area of 'buffer' meters around it.
    """
    buildings_gdf = up.classify_lu(buildings_gdfs, 'land_use') #check function for classification methodology
    buildings_gdf = up.pragmatic_score(buildings_gdf, radius = 200)
    
    # global landmarkness components weights
    g_cW = {'vScore': 0.00, 'sScore' : 0.60, 'cScore': 0.10, 'pScore': 0.30}
    # global landmarkness indexes weights, cScore and pScore have only 1 index each
    g_iW = {'3dvis': 0.00, 'fac': 0.00, 'height': 0.00, 'area': 0.25, '2dvis': 0.35, 'neigh': 0.20 , 'road': 0.20}
    buildings_gdf = up.compute_global_score(buildings_gdf, g_cW, g_iW)
    
    return buildings_gdf

def landmark_presence(route, buildings, threshold): 
    
    # better to use not rescaled gScore values as different cities are tested
    landmarks = buildings[buildings.gScore >= threshold]
    
    
    # if route travelling towards a salient feature then distance to feature is reduced

### Elevation and Slope

In [ ]:
def elev():

    # use API to extract elevation for each node
    for _, node in node_geo.iterrows():    
        
        if 'elev' in node_geo.columns and node_geo.loc[[_], 'elev'].iloc[0] > 0: continue # if elev already extracted then skip
        
        x, y = node['geometry'].x, node['geometry'].y
        r_str = 'https://api.open-elevation.com/api/v1/lookup?locations=%s,%s' % (y, x)  # as of 11.2019, API unreliable
        
        try:
            r = requests.get(r_str)
        except requests.exceptions.RequestException as e:
            print ('Request error', e)
            continue
        
        elev = 0
        if r.status_code == 200:
            elev = int(r.json()['results'][0]['elevation'])
        else:
            print (r_str)
            print (_, 'elevation not found')
        
        node_geo.loc[[_],  'elev'] = elev

In [ ]:
def slope(threshold, pc_inc):
    # adjust distance where variation in elevation between u and v
    
    for _, road in road_geo.iterrows():
        
        start_elev = node_geo['elev'].loc[[road['u']]].values[0]
        end_elev = node_geo['elev'].loc[[road['v']]].values[0]
        
        slope = ((start_elev - end_elev) / road['length']) * 100
      
        if slope > threshold or slope < -threshold:
            #print (road['name'], slope)
            curr_distance = road_geo['new_length'].loc[[_]].values[0]
            road_geo.loc[[_],  'new_length'] = curr_distance * pc_inc

### Land Use

In [ ]:
def land_use(buff_dist, pc_inc):
    # proximity to shops/activities leads to distraction and distance overestimation
    
    road_geo_buff = road_geo.copy()
    road_geo_buff = road_geo_buff.to_crs({'init': city_epsg})
    road_geo_buff['geometry'] = road_geo_buff.apply(buffer, args=[buff_dist], axis=1) # ~25m buffer set up around roads
    
    for _, road in road_geo_buff.iterrows():
        features = 0
        features += buildings.loc[buildings['amenity'].notna()].intersects(road['geometry']).values.sum()
        
        if features >= 3: 
            curr_distance = road_geo['new_length'].loc[[_]].values[0]
            road_geo.loc[[_],  'new_length'] = curr_distance * pc_inc


### Intersections and Turns

In [ ]:
def intersections(route):
    # get junction-based features of the route
    p1 = []
    p2 = []
    p3 = []

    minor_junc = 0
    medium_junc = 0
    major_junc = 0
    roundabouts = 0
    
    turns = 0
    sum_deviation = 0
        
    i = 0
    
    for _, row in route.iterrows():  
                
        # counts of roundabouts
        if node_geo.loc[[row['v']]].highway.iloc[0] == 'mini_roundabout' or node_geo.loc[[row['v']]].highway.iloc[0] == 'turning_circle':
            roundabouts += 1
        
        # get road grades at junction
        if len(route) > i+1: connector = route.iloc[[i+1]] # gets next road
        else: i+=1; continue
        
        if connector.name.iloc[0] != row[8]: # if change in road
            if connector['highway'].iloc[0] == 'motorway' or connector['highway'].iloc[0] == 'trunk' or connector['highway'].iloc[0] == 'trunk_link':
                major_junc += 1
            elif connector['highway'].iloc[0] == 'primary':
                medium_junc += 1
            elif connector['highway'].iloc[0] == 'secondary':
                minor_junc += 1
        i+=1  
        
        # turns and total angular deviation
        long = row['geometry'].xy[0][0]
        lat = row['geometry'].xy[1][0]
        p3 = [long, lat]

        if p2 == [] or p1 == p2:
            p2 = p3
            if p1 == []:
                p1 = p2
            continue

        # Convert the points to numpy latitude/longitude radians space
        a = np.radians(np.array(p1))
        b = np.radians(np.array(p2))
        c = np.radians(np.array(p3))

        avec = a - b
        cvec = c - b

        lat = b[0]
        avec[1] *= math.cos(lat)
        cvec[1] *= math.cos(lat)

        angle2deg = angle_between_vectors_degrees(avec, cvec)
        deviation = 180-angle2deg
        sum_deviation += deviation

        p1 = p2
        p2 = p3
        
        if deviation > 60:
            turns +=1
            
    return minor_junc, medium_junc, major_junc, roundabouts, turns, sum_deviation


### Length

In [ ]:
def length_calcs(route, origin, destination):
    # get length-based features of route
    length = 0
    cog_length = 0
    euclid = 0
        
    # get total length
    for _, row in route.iterrows():  
        cog_length += row['new_length']
        length += row['length']

    # get geoms and convert to coordinate system to handle metres easily
    o_geom = node_geo.loc[[origin]].geometry
    d_geom = node_geo.loc[[destination]].geometry
    o_geom = o_geom.to_crs({'init': city_epsg})
    d_geom = d_geom.to_crs({'init': city_epsg})
    
    # get euclidean distance
    euclid = o_geom.iloc[0].distance(d_geom.iloc[0])
    
    return length, cog_length, euclid

### Network Configuration

In [ ]:
def network_complexity(route):
    # sum of nodes within radius of each node on route

    sum_node = 0
    
    for _, row in route.iterrows():  
        search_geom = node_geo.loc[[row['v']]].geometry
        search_geom = search_geom.to_crs({'init': city_epsg})
        search_geom = search_geom.buffer(buffer_dist) # make buffer region at end of segment
        sum_node += node_geo.intersects(search_geom.unary_union).values.sum() - 1 # search and sum other nodes (subtract 1 for current node)

    if sum_node > 0: return sum_node
    else: return 0

### Route Prominence

In [ ]:
def prominence(mode):
    # relative to travel speed
    
    for _, road in road_geo.iterrows():
        if mode == 'drive':
            if road['highway'] == 'motorway' or road['highway'] == 'trunk':
                curr_distance = road_geo['new_length'].loc[[_]].values[0]
                road_geo.loc[[_],  'new_length'] = curr_distance * 0.7
            if road['highway'] == 'primary':
                curr_distance = road_geo['new_length'].loc[[_]].values[0]
                road_geo.loc[[_],  'new_length'] = curr_distance * 0.8
            if road['highway'] == 'secondary':
                curr_distance = road_geo['new_length'].loc[[_]].values[0]
                road_geo.loc[[_],  'new_length'] = curr_distance * 0.9
            if road['highway'] == 'footway' or road['highway'] == 'service':
                curr_distance = road_geo['new_length'].loc[[_]].values[0]
                road_geo.loc[[_],  'new_length'] = curr_distance * 2.0
        elif mode == 'walk':
            if road['highway'] == 'footway' or road['highway'] == 'service':
                curr_distance = road_geo['new_length'].loc[[_]].values[0]
                road_geo.loc[[_],  'new_length'] = curr_distance * footpath_effect


## Global City Analysis

This section implements the methods for estimating cognitive distance and applies them in 16 cities. This process is divided into four steps:

1. Setup parameters for the study - including defining the effect of each spatial factor, the test settings, and the location of test cities.
2. Iterate over the study cities, loading the road network and buildings for the study region and making initial 'road-based' adjustments to the GIS data.
3. In `execute_paths`, randomly select origin and destination nodes, and request Euclidean distances, metric path distances, and cognitive distances.
4. All path-based calculations are made in `calculate_distance`, where the majority of 'route-based' adjustments are executed. 

Additional cities can be tested by adding them to the `test_cities.txt` file, maintaining the same format structure.

### Set parameters

#### Estimate parameters
These current settings are in line with those described in the paper.

In [ ]:
footpath_effect = 0.9
land_use_effect = 1.2
slope_effect = 1.5
minor_junc_effect = 0.01
medium_junc_effect = 0.02
major_junc_effect = 0.05
turn_effect = 0.1
config_effect = 0.005
near_prox_effect = 0.8
near_prox_threshold = 200
far_prox_effect = 1.2
far_prox_threshold = 800

buffer_dist = 25 # buffer distance (in metres) used for proximity calculations
slope_threshold = 5.0 # absolute change in slope required for effect

#### Test settings

In [ ]:
n_paths = 10 # number of paths to test per city
test_radius = 1000 # radius (in metres) of network from city centroid
include_elev = False # option to include elevation in calcs
travel_mode = 'walk' # 'drive' or 'walk'

#### Test cities data

In [65]:
cities_file_path = 'input/test_cities.txt' # location of city centroids and EPSG information
cities = pd.read_csv(cities_file_path, delimiter='\t') # load cities data

### Run analysis on cities

In [68]:
all_records = []
for _, row in cities.iterrows():

    city_epsg = 'epsg:' + str(row['epsg'])
    print (row['City'])

    # get the road network and buildings for given city at given radius from centre point
    try:
        road_network = ox.graph_from_point([row['Lat'], row['Lon']], distance = test_radius, network_type=travel_mode, simplify=False, clean_periphery=True)
        
#         buildings = ox.footprints.footprints_from_point([row['Lat'], row['Lon']], distance = test_radius, retain_invalid=False)
    except:
        pass # to ignore TopologyException 
    
    road_network = ox.simplify_graph(road_network)
#     buildings = buildings.to_crs({'init': city_epsg}) #assigned within landmark function
    
    # extract geography - both nodes and roads
    node_geo, road_geo = ox.graph_to_gdfs(road_network, nodes=True, edges=True, fill_edge_geometry=True)
    node_geo = node_geo.to_crs({'init': city_epsg})
    road_geo = road_geo.to_crs({'init': city_epsg})
    road_geo['new_length'] = road_geo['length']
    # assigning landmarks scores
    buildings = detect_landmarks(row['City'], road_geo, test_radius)
    
    # apply weights to road segments
    if include_elev:
        print ('  Extracting elevation')
        elev() # calculates elevations of each node using API
        print ('  Calculating slope')
        slope(slope_threshold, slope_effect) # calculates slope and increases distance above threshold
    print ('  Making land use adjustments')
    land_use(buffer_dist, land_use_effect) # increases distance where there are intervening opps
    print ('  Adjusting according to road grade')
    prominence(travel_mode) # adjust length of major roads
    
    print ('  Executing paths')
    city_records = execute_paths(row['City'], city_epsg)
    
    if len(all_records) == 0: all_records = city_records[:]
    else: all_records.extend(city_records)

# put all records into a df
city_distance_calcs = pd.DataFrame(all_records, columns=['city','euclid','network','cognitive','f_segment','f_isections','f_turns','f_config','f_locations'])
    

New York City - Tribeca


NameError: name 'road_network' is not defined

### Origin-destination selection method

In [ ]:
def execute_paths(this_city, epsg): 
    
    city_records = []
    city_count = 0
    while city_count < n_paths:

        start = road_network.nodes[int(node_geo.sample(1)['osmid'].iloc[0])]
        start_nx = start['osmid']
        end = road_network.nodes[int(node_geo.sample(1)['osmid'].iloc[0])]
        end_nx = end['osmid']

        # reproject points to local reference
        p1 = Point(start['x'], start['y'])
        p2 = Point(end['x'], end['y'])
        project = partial(pyproj.transform, 
            pyproj.Proj(init = 'epsg:4326'), # source coordinate system (WGS84)
            pyproj.Proj(init = city_epsg)) # destination coordinate system 

        start_proj = transform(project, p1)  # apply projection
        end_proj = transform(project, p2)  # apply projection
        
        # calculate distances
        euclid_dist = start_proj.distance(end_proj)
        cognitive_dist, network_dist, f1, f2, f3, f4, f5 = calculate_distance(start_nx, end_nx)
        
        if cognitive_dist > -1 and network_dist > -1: # error check
            city_records.append([this_city, euclid_dist, network_dist, cognitive_dist, f1, f2, f3, f4, f5])
            city_count +=1
        
        # else: repeat for another OD pair
   
    return city_records

### Path and distance calculations

In [ ]:
def calculate_distance(start_nx, end_nx):
    
    try:
        metric_route = nx.dijkstra_path(road_network, start_nx, end_nx, weight='length')
        cognitive_route = nx.dijkstra_path(road_network, start_nx, end_nx, weight='new_length')
    except:
        print ('   ERROR: Path computation failure')
        return -1, -1, -1, -1, -1, -1, -1

    # metric length
    route_nodes = list(zip(metric_route[:-1], metric_route[1:]))
    index = [road_geo[(road_geo['u']==u) & (road_geo['v']==v)].index[0] for u, v in route_nodes]
    route_geo = road_geo.loc[index]
    metric_length, _, _ = length_calcs(route_geo, start_nx, end_nx) 

    # cognitive length
    route_nodes = list(zip(cognitive_route[:-1], cognitive_route[1:]))
    index = [road_geo[(road_geo['u']==u) & (road_geo['v']==v)].index[0] for u, v in route_nodes]
    route_geo = road_geo.loc[index]
    _, cog_length, euclid = length_calcs(route_geo, start_nx, end_nx)
    
    f1 = cog_length - metric_length  #  road segment effects
    cog_len_prev = cog_length
    
    # intersections and turns 
    minor_junc, medium_junc, major_junc, roundabouts, turns, sum_deviation = intersections(route_geo)
    
    cog_length *= 1 + (minor_junc * minor_junc_effect)
    cog_length *= 1 + (medium_junc * medium_junc_effect)
    cog_length *= 1 + ((major_junc + roundabouts) * major_junc_effect)
    f2 = cog_length - cog_len_prev  # intersection effect
    cog_len_prev = cog_length
    
    cog_length *= 1 + (turns * turn_effect)
    f3 = cog_length - cog_len_prev  # turn effect
    cog_len_prev = cog_length

    # network complexity
    sum_nodes = network_complexity(route_geo)
    cog_length *= 1 + (sum_nodes * config_effect)
    f4 = cog_length - cog_len_prev  # network effect
    cog_len_prev = cog_length

    # context specific relating to proximity of destination
    if euclid < near_prox_threshold: cog_length *= near_prox_effect
    elif euclid > far_prox_threshold: cog_length *= far_prox_effect
    f5 = cog_length - cog_len_prev  # distribution effect

    return cog_length, metric_length, f1, f2, f3, f4, f5


## Aggregation and Export

Some simple methods for summarising and exporting the results.

In [ ]:
summary_table = pd.DataFrame()
summary_table['euclid'] = city_distance_calcs.groupby('city')['euclid'].mean()
summary_table['network'] = city_distance_calcs.groupby('city')['network'].mean()
summary_table['cognitive'] = city_distance_calcs.groupby('city')['cognitive'].mean()
summary_table['f_segment'] = city_distance_calcs.groupby('city')['f_segment'].mean()
summary_table['f_isections'] = city_distance_calcs.groupby('city')['f_isections'].mean()
summary_table['f_turns'] = city_distance_calcs.groupby('city')['f_turns'].mean()
summary_table['f_config'] = city_distance_calcs.groupby('city')['f_config'].mean()
summary_table['f_locations'] = city_distance_calcs.groupby('city')['f_locations'].mean()
summary_table

In [ ]:
filename = 'output/results_n' + str(n_paths) + '_r' + str(test_radius) + '_' + travel_mode + '.csv'
summary_table.to_csv(filename)

### Utility functions

In [ ]:
def buffer(row, value):
     return row.geometry.buffer(value)

In [ ]:
def angle_between_vectors_degrees(u, v):
    try:
        angle = np.degrees(math.acos(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))))
        return angle
    except:
        print ('   WARNING: No angle between vectors detected')
        return 0

In [ ]:
def dist_features(a,b):
    return features.loc[[a]].geometry.centroid.iloc[0].distance(features.loc[[b]].geometry.centroid.iloc[0])